In [1]:
import numpy as np
import cv2
from src import forward_energy, seam_carving
import skvideo
import skvideo.io

In [2]:
# yi po shi
cut = []
input_file = []
fps = []
cap = []
frame_count = []
vid_out = []

In [3]:
from tkinter import *
from tkinter import messagebox
from tkinter import filedialog
from tkinter.ttk import *
from tkinter.ttk import Progressbar
import os

In [4]:
window = Tk()
window.title("VideoSeamCarving")
style = Style()
style.theme_use('default')
style.configure("black.Horizontal.TProgressbar", background='black')

In [5]:
# set up frame
frame = Frame(window, width=1280, height=720)
frame.grid(row=0, column=0, sticky="NW")
frame.grid_propagate(0)
frame.update()

In [6]:
# add title
lbl_title = Label(frame, text="Video Seam Carving",
                  font=("Helvetica Bold", 25))
lbl_title.place(x=640, y=25, anchor="center")  # set position

In [7]:
# cut type selection
# label
lbl_cut = Label(frame, text="Select Cut Type", font=("Helvetica Bold", 15))
lbl_cut.place(x=300, y=100, anchor="center")
# combo box
cut_type = Combobox(frame)
cut_type['values'] = ("horizontal", "vertical")
cut_type.current(0)  # set the selected item
cut_type.place(x=300, y=125, anchor="center")

In [8]:
# cut number selection
# label
lbl_cut2 = Label(frame, text="Input Seam Number", font=("Helvetica Bold", 15))
lbl_cut2.place(x=400, y=100, anchor="center")

# entry
cut_number = Entry(frame, width=10)
cut_number.place(x=400, y=125, anchor="center")

# handle set_cut event
def set_cut():
    cut.append(cut_type.get())
    cut.append(int(cut_number.get()))
    # message box
    messagebox.showinfo('cut parameter', f'Cut is set to {cut}!')


# add button
btn_set = Button(frame, text="Cut set!", command=set_cut)
btn_set.place(x=600, y=125, anchor="center")  # button position

In [9]:
# input section
# label
lbl_input = Label(frame, text="Select File", font=("Helvetica Bold", 15))
lbl_input.place(x=300, y=300, anchor="center")

# handle input_file event


def select_file():
    # file = filedialog.askopenfilename(filetypes=(("MOV Files", "*.mov"),("MP4 Files", "*.mp4"),("AVI Files","*.avi"))) # after choosing, file will hold the path
    # after choosing, file will hold the path
    file = filedialog.askopenfilename(filetypes=(
        ("MP4 Files", "*.mp4"), ("MOV Files", "*.mov"), ("M4V Files", "*.m4v")))
    input_file.append(file)


def test():
    #cap = cv2.VideoCapture(input_file[0])
    cap.append(cv2.VideoCapture(input_file[0]))
    #fps = cap.get(cv2.CAP_PROP_FPS)
    fps.append(cap[0].get(cv2.CAP_PROP_FPS))
    #frame_count = int(cap[0].get(cv2.CAP_PROP_FRAME_COUNT))
    frame_count.append(int(cap[0].get(cv2.CAP_PROP_FRAME_COUNT)))
    duration = frame_count[0] / fps[0]
    #print('fps = ' + str(fps))
    #print('number of frames = ' + str(frame_count))
    #print('duration (S) = ' + str(duration))

    minutes = int(duration / 60)
    seconds = duration % 60
    #print('duration (M:S) = ' + str(minutes) + ':' + str(seconds))
    #messagebox.showinfo('input file',f'Input file is set to {input_file[0]}!')
    messagebox.showinfo('input information',
                        f'fps = {fps[0]}\n number of frames = {frame_count} \n duration (S) = {duration} \n duration (M:S) = {minutes} : {seconds}')


# add button
btn_input = Button(frame, text="Select File", command=select_file)
btn_input.place(x=300, y=350, anchor="center")

# for test only
btn_test = Button(frame, text="Input Set!", command=test)
btn_test.place(x=500, y=350, anchor="center")

In [10]:
# output section
# label
lbl_out = Label(frame, text="Output File Name", font=("Helvetica Bold", 15))
lbl_out.place(x=800, y=100, anchor="center")

# entry
output_name = Entry(frame, width=10)
output_name.place(x=800, y=125, anchor="center")

# handle output event
def output():
    vid_out.append(skvideo.io.FFmpegWriter(output_name.get(),
                                           outputdict={'-vcodec': 'libx264',
                                                       '-pix_fmt': 'yuv420p',
                                                       '-r': str(fps[0])}))
    # message box
    messagebox.showinfo(
        'Output information', f'Output file is set to {output_name.get()}!\n check fps = {str(fps[0])}')


# add button
btn_out = Button(frame, text="Output set!", command=output)
btn_out.place(x=1000, y=125, anchor="center")  # button position

In [11]:
# run section (not finished)

# handle run event
def run():
    bar = Progressbar(frame, length=200)
    bar.place(x=550, y=500)
    bar['value'] = 0  # placeholder
    i = 1

    while cap[0].isOpened():
        ret, frames = cap[0].read()
        if ret:
            #print("Carve frame: " + str(i) + "/" + str(frame_count))
            bar['value'] = i/frame_count[0]*100
            window.update_idletasks()
            img, eimg = seam_carving(frames, forward_energy, cut[1], cut[0])
            img = (img*255).astype(np.uint8)
            img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
            vid_out[0].writeFrame(img)
            cv2.imshow('frame', cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
            cv2.waitKey(int(fps[0]))
        else:
            break
        i += 1

    cap[0].release()
    vid_out[0].close()
    cv2.destroyAllWindows()


# add button
btn_run = Button(window, text="Run!", command=run)
btn_run.place(x=640, y=680, anchor="center")

In [ ]:
# run
window.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\Yuan Liang\Anaconda3\lib\tkinter\__init__.py", line 1705, in __call__
    return self.func(*args)
  File "<ipython-input-10-f4f637753670>", line 15, in output
    '-r': str(fps[0])}))
  File "C:\Users\Yuan Liang\Anaconda3\lib\site-packages\skvideo\io\ffmpeg.py", line 336, in __init__
    assert str.encode(self.extension).lower() in _FFMPEG_SUPPORTED_ENCODERS, "Unknown encoder extension: " + self.extension.lower()
AssertionError: Unknown encoder extension: .mv4
